In [4]:
import os
import sys
sys.path.append("./ddn/")
sys.path.append("./")
import warnings
warnings.filterwarnings('ignore')

import torch
import numpy as np
import scipy.special
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd

from scipy.linalg import block_diag
from torch.utils.data import Dataset, DataLoader
from utils.bernstein import bernstein_coeff_order10_new
from ddn.ddn.pytorch.node import AbstractDeclarativeNode
from utils.models.ddn import *
from utils.nodes.OPTNode import *

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from av2.datasets.motion_forecasting.data_schema import ArgoverseScenario, ObjectState, ObjectType, Track, TrackCategory
from typing import List, Tuple

def _load_tracks_from_tabular_format(tracks_df: pd.DataFrame) -> List[Track]:
    """Load tracks from tabular data format.
    Args:
        tracks_df: DataFrame containing all track data in a tabular format.
    Returns:
        All tracks associated with the scenario.
    """
    tracks: List[Track] = []

    for track_id, track_df in tracks_df.groupby("track_id"):

        observed_states: List[bool] = track_df.loc[:, "observed"].values.tolist()
        object_type: ObjectType = ObjectType(track_df["object_type"].iloc[0])
        object_category: TrackCategory = TrackCategory(track_df["object_category"].iloc[0])
        timesteps: List[int] = track_df.loc[:, "timestep"].values.tolist()
        positions: List[Tuple[float, float]] = list(
            zip(
                track_df.loc[:, "position_x"].values.tolist(),
                track_df.loc[:, "position_y"].values.tolist(),
            )
        )
        headings: List[float] = track_df.loc[:, "heading"].values.tolist()
        velocities: List[Tuple[float, float]] = list(
            zip(
                track_df.loc[:, "velocity_x"].values.tolist(),
                track_df.loc[:, "velocity_y"].values.tolist(),
            )
        )

        object_states: List[ObjectState] = []
        for idx in range(len(timesteps)):
            object_states.append(
                ObjectState(
                    observed=observed_states[idx],
                    timestep=timesteps[idx],
                    position=positions[idx],
                    heading=headings[idx],
                    velocity=velocities[idx],
                )
            )

        tracks.append(
            Track(track_id=track_id, object_states=object_states, object_type=object_type, category=object_category)
        )

    return tracks

In [6]:
from av2.datasets.motion_forecasting import scenario_serialization
from av2.datasets.motion_forecasting.viz.scenario_visualization import visualize_scenario
from av2.map.map_api import ArgoverseStaticMap

colors = {
    TrackCategory.FOCAL_TRACK: "red",
    TrackCategory.SCORED_TRACK: "yellow",
    TrackCategory.UNSCORED_TRACK: "blue",
    TrackCategory.TRACK_FRAGMENT: "grey"
}

In [7]:
import os
# os.listdir("/mnt/e/datasets/argoverse/val/")

In [8]:
rho_obs = 1.2
rho_eq = 10.0
weight_smoothness = 10

pred_len = 60
num = 60
t_fin = 6.0
a_obs = 1.0
b_obs = 1.0

tot_time = np.linspace(0.0, t_fin, num)
tot_time_copy = tot_time.reshape(num, 1)
P, Pdot, Pddot = bernstein_coeff_order10_new(10, tot_time_copy[0], tot_time_copy[-1], tot_time_copy)
nvar = np.shape(P)[1]

A_eq = np.vstack((P[0], Pdot[0], Pddot[0], P[-1], Pdot[-1], Pddot[-1]))
Q_smoothness = np.dot(Pddot.T, Pddot)

In [18]:
cnt = 0
focals = []
obstacles = []

for file in os.listdir("/mnt/e/datasets/argoverse/val"):
    cnt = cnt + 1
    tracks_df = pd.read_parquet(f"/mnt/e/datasets/argoverse/val/{file}/scenario_{file}.parquet")

    tracks = _load_tracks_from_tabular_format(tracks_df)
    scenario_id = tracks_df["scenario_id"][0]
    focal_track_id = tracks_df["focal_track_id"][0]
    city_name = tracks_df["city"][0]
    map_id = tracks_df["map_id"][0] if "map_id" in tracks_df.columns else None
    slice_id = tracks_df["slice_id"][0] if "slice_id" in tracks_df.columns else None
    
    
    timestamps_ns = np.linspace(
        tracks_df["start_timestamp"][0], tracks_df["end_timestamp"][0], num=tracks_df["num_timestamps"][0]
    )

    scenario = ArgoverseScenario(
            scenario_id=scenario_id,
            timestamps_ns=timestamps_ns,
            tracks=tracks,
            focal_track_id=focal_track_id,
            city_name=city_name,
            map_id=map_id,
            slice_id=slice_id,
    )

    obstacle = []
    # plt.figure(figsize=(10,10))
    for track in scenario.tracks:
        x = [i.position[0] for i in track.object_states]
        y = [i.position[1] for i in track.object_states]
        # plt.scatter(x,y, color=colors[track.category])    
        if track.category in [TrackCategory.FOCAL_TRACK, TrackCategory.SCORED_TRACK]:
            if len(x) != 110:
                print(len(x), len(y))
        if track.category in [TrackCategory.FOCAL_TRACK]:
            focals.append([x, y])
        elif track.category in [TrackCategory.SCORED_TRACK]:
            obstacle.append([x, y])
            pass
    obstacles.append(obstacle)
    # plt.axis('equal')
    if cnt > 100: break

In [19]:
fixed_params = []
variable_params = []
for focal in focals:
    

(101, 2, 110)

In [ ]:

x_obs_temp = np.hstack((-10000.0, 10000.79, 30000.0, 10000.0))
y_obs_temp = np.hstack((-10000.0, 10000.0, -30000.80, 10000.0))

num_obs = np.shape(x_obs_temp)[0]

x_obs = np.ones((num_obs, num)) * x_obs_temp[:, np.newaxis]
y_obs = np.ones((num_obs, num)) * y_obs_temp[:, np.newaxis]

obs_inp_list = []
for i in range(num_obs):
    obs_inp_list.extend([x_obs_temp[i], y_obs_temp[i], a_obs])
    
A_obs = np.tile(P, (num_obs, 1))
